In [9]:
prompt = """You are an expert in categorising GP surgery feedback.
I want you to classify the patient comments into one of the categories:
       
        "Staff Professionalism",
        "Communication Effectiveness",
        "Appointment Availability",
        "Waiting Time",
        "Facility Cleanliness",
        "Patient Respect",
        "Treatment Quality",
        "Staff Empathy and Compassion",
        "Administrative Efficiency",
        "Reception Staff Interaction",
        "Environment and Ambiance",
        "Follow-up and Continuity of Care",
        "Accessibility and Convenience",
        "Patient Education and Information",
        "Feedback and Complaints Handling",
        "Test Results",
        "Surgery Website",
        "Telehealth",
        "Vaccinations",
        "Prescriptions and Medication Management",
        "Mental Health Support",
   

       

        Patient Feedback: 
        
        Please get rid off the dead chair lift at the surgery"""


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "refuelai/Llama-3-Refueled"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

/Users/janduplessis/.pyenv/versions/3.10.6/envs/notebooks/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
messages = [{"role": "user", "content": prompt}]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("mps")

outputs = model.generate(inputs, max_new_tokens=5)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You are an expert in categorising GP surgery feedback.
I want you to classify the patient comments into one of the categories:
       
        "Staff Professionalism",
        "Communication Effectiveness",
        "Appointment Availability",
        "Waiting Time",
        "Facility Cleanliness",
        "Patient Respect",
        "Treatment Quality",
        "Staff Empathy and Compassion",
        "Administrative Efficiency",
        "Reception Staff Interaction",
        "Environment and Ambiance",
        "Follow-up and Continuity of Care",
        "Accessibility and Convenience",
        "Patient Education and Information",
        "Feedback and Complaints Handling",
        "Test Results",
        "Surgery Website",
        "Telehealth",
        "Vaccinations",
        "Prescriptions and Medication Management",
        "Mental Health Support",
   

       

        Patient Feedback: 
        
        Please get rid off th

In [11]:
import pandas as pd

In [12]:
data = pd.read_csv('data.csv')
data.head(2)

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id
0,2023-07-13 15:28:00,Extremely likely,Very friendly caring staff treated me with res...,NaN,Earls-Court-Surgery,Brompton-Health-PCN,positive,0.892932,neutral,0.0,13.0,3,5.0,Staff Empathy and Compassion,NaN,NaN,NaN
1,2023-07-13 15:29:00,Extremely likely,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,4.0,0,5.0,NaN,NaN,NaN,NaN


In [24]:
def categorize_feedback(text):
    prompt = f"""You are an expert in categorising GP surgery feedback.
        I want you to classify the patient comments into one of the categories:
               
                "Staff Professionalism",
                "Communication Effectiveness",
                "Appointment Availability",
                "Waiting Time",
                "Facility Cleanliness",
                "Patient Respect",
                "Treatment Quality",
                "Staff Empathy and Compassion",
                "Administrative Efficiency",
                "Reception Staff Interaction",
                "Environment and Ambiance",
                "Follow-up and Continuity of Care",
                "Accessibility and Convenience",
                "Patient Education and Information",
                "Feedback and Complaints Handling",
                "Test Results",
                "Surgery Website",
                "Telehealth",
                "Vaccinations",
                "Prescriptions and Medication Management",
                "Mental Health Support",
        
                Patient Feedback: {text}"""

    messages = [{"role": "user", "content": prompt}]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("mps")

    outputs = model.generate(inputs, max_new_tokens=5)
    return tokenizer.decode(outputs[0])

In [25]:
free_text_list = data['free_text'].to_list()

In [26]:
free_text_list

['Very friendly caring staff treated me with respect and listened to my problem',
 nan,
 'The doctor was very understanding and very knowledgeable',
 nan,
 'I was spoken to politely on the phone and a telephone appointment arranged with my doctor which due to a mistake on a locked phone I wasn t able to answer but I was called back twice to complete my assessment of my health worry',
 nan,
 'Very professional and very kind I feel comfortable when I attend my appointment there',
 nan,
 nan,
 'The nurse Miss PERSON was absolutely amazing understanding kind fun thorough careful She is the best',
 'I have been with surgery for considerable years and was well looked after by Dr PERSON hoping the work ethic continues apart that good service good location etc',
 nan,
 nan,
 'Because the staff are very helpful',
 'Nurse PERSON is really helpful and look after us',
 nan,
 nan,
 'The receptionist was really very very helpful',
 nan,
 nan,
 nan,
 nan,
 'All the colleagues are nice and professiona

In [28]:
from tqdm import tqdm

output_list = []

for l in tqdm(free_text_list):
    if str(l) != 'nan':
        output_list.append(categorize_feedback(str(l)))
    elif str(l) == 'nan':
        output_list.append('')
        

  0%|                                                                               | 0/4740 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|                                                                  | 1/4740 [01:55<151:33:31, 115.13s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|                                                                    | 3/4740 [03:47<93:53:08, 71.35s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable

KeyboardInterrupt: 

In [50]:
output_list[113].split()[-6:]

['and',
 'listen',
 'to',
 'them<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
 'Staff',
 'Professionalism<|eot_id|><|end_of_text|>']

In [51]:
new_data = {
    "output": output_list 
}
new = pd.DataFrame(new_data)

In [52]:
new

,output
0,<|begin_of_text|><|start_header_id|>user<|end_...
1,
2,<|begin_of_text|><|start_header_id|>user<|end_...
3,
4,<|begin_of_text|><|start_header_id|>user<|end_...
...,...
158,
159,<|begin_of_text|><|start_header_id|>user<|end_...
160,<|begin_of_text|><|start_header_id|>user<|end_...
161,<|begin_of_text|><|start_header_id|>user<|end_...


In [53]:
new.to_csv('new.csv', index=True)

In [55]:
load  = pd.read_csv('data.csv')
load.head(2)

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,output
0,13/07/2023 15:28,Extremely likely,Very friendly caring staff treated me with res...,NaN,Earls-Court-Surgery,Brompton-Health-PCN,positive,0.892932,neutral,0.0,13.0,3,5.0,Staff Empathy and Compassion,NaN,NaN,NaN,<|begin_of_text|><|start_header_id|>user<|end_...
1,13/07/2023 15:29,Extremely likely,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,4.0,0,5.0,NaN,NaN,NaN,NaN,NaN


In [73]:
new_list = []
for index, row in load.iterrows():
    output = row['output']
    if str(output) != 'nan':
        new_list.append(str(output)[-80:])
    elif str(output) == 'nan':
        new_list.append('')
        

In [74]:
new_list

['eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nStaff Empathy and Compass',
 '',
 'eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nStaff Empathy and Compass',
 '',
 'r_id|>assistant<|end_header_id|>\n\nStaff Professionalism<|eot_id|><|end_of_text|>',
 '',
 'r_id|>assistant<|end_header_id|>\n\nStaff Professionalism<|eot_id|><|end_of_text|>',
 '',
 '',
 'eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nStaff Empathy and Compass',
 'd|><|start_header_id|>assistant<|end_header_id|>\n\nFacility Cleanliness<|eot_id|>',
 '',
 '',
 'eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nStaff Empathy and Compass',
 'eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nStaff Empathy and Compass',
 '',
 '',
 'art_header_id|>assistant<|end_header_id|>\n\nReception Staff Interaction<|eot_id|>',
 '',
 '',
 '',
 '',
 'r_id|>assistant<|end_header_id|>\n\nStaff Professionalism<|eot_id|><|end_of_text|>',
 '',
 '',
 'r_id|>assistant<|end_header_id|>\n\nStaff Professiona

In [75]:
new_data = {
    "new_list": new_list
}
new_data = pd.DataFrame(new_data)

In [76]:
new_data.to_csv('new_list.csv')